In [1]:
import os
import pandas as pd
import sqlite3

In [2]:
raw_path = "data/raw/NVDA.csv"
processed_dir = "data/processed"
os.makedirs(processed_dir, exist_ok=True)
sqlite_path = os.path.join(processed_dir, "NVDA_proc.sqlite")

# load CSV
df = pd.read_csv(raw_path)

# If 'Date' column is missing, try to detect a date-like column and create 'Date'
if 'Date' not in df.columns:
    date_col = None
    max_parsed = -1
    date_parsed = None
    for col in df.columns:
        # try parsing common ISO date strings first
        parsed = pd.to_datetime(df[col], errors='coerce')
        num_parsed = parsed.notna().sum()
        # if nothing parsed, try milliseconds and seconds timestamps
        if num_parsed == 0:
            parsed_ms = pd.to_datetime(df[col], unit='ms', errors='coerce')
            parsed_s = pd.to_datetime(df[col], unit='s', errors='coerce')
            if parsed_ms.notna().sum() >= parsed_s.notna().sum():
                parsed = parsed_ms
            else:
                parsed = parsed_s
            num_parsed = parsed.notna().sum()
        if num_parsed > max_parsed:
            max_parsed = num_parsed
            date_col = col
            date_parsed = parsed
    if max_parsed > 0:
        df['Date'] = date_parsed
    else:
        raise KeyError("Date column not found and no column could be parsed as dates. Check CSV format.")

# Ensure 'Date' is datetime (try without unit first, then with 'ms' and 's' if necessary)
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
if df['Date'].isna().sum() > len(df) * 0.5:
    df['Date'] = pd.to_datetime(df['Date'], unit='ms', errors='coerce')
    if df['Date'].isna().sum() > len(df) * 0.5:
        df['Date'] = pd.to_datetime(df['Date'], unit='s', errors='coerce')

# Drop rows with invalid dates or missing values
df_clean = df.dropna().reset_index(drop=True)

# Write to SQLite (table name: NVDA); datetime will be stored as TEXT in ISO format
with sqlite3.connect(sqlite_path) as conn:
    df_clean.to_sql("NVDA", conn, if_exists="replace", index=False)

print(f"Cleaned data saved to {sqlite_path} ({len(df_clean)} rows).")

C:\Users\soltv\AppData\Local\Temp\ipykernel_7812\605905016.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed = pd.to_datetime(df[col], errors='coerce')
C:\Users\soltv\AppData\Local\Temp\ipykernel_7812\605905016.py:16: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  parsed = pd.to_datetime(df[col], errors='coerce')
C:\Users\soltv\AppData\Local\Temp\ipykernel_7812\605905016.py:20: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  parsed_ms = pd.to_datetime(

Cleaned data saved to data/processed\NVDA_proc.sqlite (6517 rows).


C:\Users\soltv\AppData\Local\Temp\ipykernel_7812\605905016.py:20: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  parsed_ms = pd.to_datetime(df[col], unit='ms', errors='coerce')
C:\Users\soltv\AppData\Local\Temp\ipykernel_7812\605905016.py:21: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  parsed_s = pd.to_datetime(df[col], unit='s', errors='coerce')
